In [22]:
import tenseal as ts
import utils
import pandas as pd

In [23]:
context= ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[40, 21, 40])
context.generate_galois_keys()
context.global_scale = 2**21  # Échelle réduite pour plus de stabilité

# Sauvegarde des clés
secret_context = context.serialize(save_secret_key=True)

In [24]:
context.generate_galois_keys()
context.global_scale = 2**40
secret_context = context.serialize(save_secret_key=True)
utils.write_data("keys/secret_key.txt", secret_context)

In [25]:
context.make_context_public()
public_context = context.serialize()
utils.write_data("keys/public_key.txt", public_context)

# Cryptage des données sensibles :

In [34]:
context=ts.context_from(utils.read_data("keys/secret_key.txt"))
data = pd.DataFrame({
   "Age": [45.0, 34.0, 67.0, 52.0, 29.0],
    "Pression": [120, 135, 140, 110, 130],
    "Cardiaque": [80, 76, 85, 90, 72],
    "Température": [36.5, 37.0, 38.2, 36.7, 37.5]
})

In [30]:
age_crypted = ts.ckks_vector(context, data.Age.tolist())
pression_crypted = ts.ckks_vector(context, data.Pression.tolist())
cardiaque_crypted = ts.ckks_vector(context, data.Cardiaque.tolist())
temp_crypted = ts.ckks_vector(context, data.Température.tolist())
age_crypted=ts.ckks_vector(context, data.Age.tolist())
utils.write_data("outputs_client/age_crypted.txt", age_crypted.serialize())
utils.write_data("outputs_client/pression_crypted.txt", pression_crypted.serialize())
utils.write_data("outputs_client/cardiaque_crypted.txt", cardiaque_crypted.serialize())
utils.write_data("outputs_client/temp_crypted.txt", temp_crypted.serialize())

In [35]:
m_proto = utils.read_data("outputs_client/age_crypted.txt")
m = ts.lazy_ckks_vector_from(m_proto)
m.link_context(context)
decrypted = m.decrypt()
print("Valeurs d'âge décryptées:", [round(x, 2) for x in decrypted])


Valeurs d'âge décryptées: [45.0, 34.0, 67.0, 52.0, 29.0]
